In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.formula.api import logit
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

/Users/christams/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
QB_Path = 'Cleaned_QB.csv'
WR_Path = 'Cleaned_WR.csv'
RB_Path = 'Cleaned_RB.csv'
TE_Path = 'Cleaned_TE.csv'

clean_QB = pd.read_csv(QB_Path, sep = ";")
clean_WR = pd.read_csv(WR_Path, sep = ";")
clean_RB = pd.read_csv(RB_Path, sep = ";")
clean_TE = pd.read_csv(TE_Path, sep = ";")


In [3]:
def wr_week_one(data, team, opp, players, year, week):
    
    name_options = data.Name_.unique()
    temp_players = []
    
    for people in players:
        if people in name_options:
            temp_players.append(people)
    
    players = temp_players
    
    summary_team_stats = pd.DataFrame(data={"Last_Off_Total_YDS" : [0], 
                                            "Last_Off_Total_YDS/G" : [0], 
                                            "Last_Off_Passing_YDS" : [0],
                                            "Last_Off_Passing_YDS/G" : [0], 
                                            "Last_Off_Rushing_YDS" : [0], 
                                            "Last_Off_Rushing_YDS/G" : [0], 
                                            "Last_Off_Points_PTS" : [0], 
                                            "Last_Off_PointsPTS/G" : [0], 
                                            "Last_Def_Total_YDS" : [0],
                                            "Last_Def_Total_YDS/G" : [0], 
                                            "Last_Def_Passing_YDS" : [0], 
                                            "Last_Def_Passing_YDS/G" : [0], 
                                            "Last_Def_Rushing_YDS" : [0], 
                                            "Last_Def_Rushing_YDS/G" : [0], 
                                            "Last_Def_Points_PTS" : [0],
                                            "Last_Def_PointsPTS/G" : [0]})
    
    temp = pd.DataFrame(data = {"Name_":players})
    drop = ["Name_", "Tm", "1", "Opp", "Week", "Year", "Day", 
            'Tie?','Score-not-counted', "Weight(lbs)", "Inches", "Feet",
           "Last_Off_Total_YDS", "Last_Off_Total_YDS/G", "Last_Off_Passing_YDS",
           "Last_Off_Passing_YDS/G", "Last_Off_Rushing_YDS", "Last_Off_Rushing_YDS/G", 
           "Last_Off_Points_PTS", "Last_Off_PointsPTS/G", "Last_Def_Total_YDS",
           "Last_Def_Total_YDS/G", "Last_Def_Passing_YDS", "Last_Def_Passing_YDS/G", 
           "Last_Def_Rushing_YDS", "Last_Def_Rushing_YDS/G", "Last_Def_Points_PTS",
           "Last_Def_PointsPTS/G"]
    #temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    used_data = data.drop(columns = drop)
    
    for col in used_data.columns:
            if used_data[col].dtype != object:
                #temp[col + "_Count"] = 0
                temp[col + "_Average"] = 0.0
                temp[col + "_STD"] = 0.0
                temp[col + "_Min"] = 0.0
                temp[col + "_25%"] = 0.0
                temp[col + "_50%"] = 0.0
                temp[col + "_75%"] = 0.0
                temp[col + "_Max"] = 0.0
                
                
                temp["History_Opp_" + col + "_Count"] = 0.0
                temp["History_" + col + "_Average"] = 0.0
                temp["History_" + col + "_STD"] = 0.0
                temp["History_" + col + "_Min"] = 0.0
                temp["History_" + col + "_25%"] = 0.0
                temp["History_" + col + "_50%"] = 0.0
                temp["History_" + col + "_75%"] = 0.0
                temp["History_" + col + "_Max"] = 0.0
    
    
    
    seperate_df = list(summary_team_stats.columns)
    
    #temp["Feet"] = 0
    #temp["Inches"] = 0
    #temp["Weight(lbs)"] = 0
    temp["Actaul_Dk_Points"] = 0    
                
    #History of all games per player on team   
    index = 0
    for name in players:
        indiv_player = data[(data["Name_"] == name) & (data["Year"] < year)]
        indiv_player = indiv_player.drop(columns = drop)
       
        
        for col in indiv_player:
            if seperate_df.count(col) == 0:
                stats = indiv_player[col].describe()
                #temp[col + "_Count"][index] = stats[0]
                temp[col + "_Average"][index] = float(stats[1])
                temp[col + "_STD"][index] = float(stats[2])
                temp[col + "_Min"][index] = float(stats[3])
                temp[col + "_25%"][index] = float(stats[4])
                temp[col + "_50%"][index] = float(stats[5])
                temp[col + "_75%"][index] = float(stats[6])
                temp[col + "_Max"][index] = float(stats[7])   
        index += 1
    
    
    #History of all players on team vs oppenant
    index = 0 
    for name in players:
        last_games = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] < year)]
        last_games = last_games.drop(columns = drop)
        
        for col in last_games:
            if seperate_df.count(col) == 0:
                stats = last_games[col].describe()
                #temp["History_Opp_" + col + "_Count"][index] = stats[0]
                temp["History_" + col + "_Average"][index] = float(stats[1]) 
                temp["History_" + col + "_STD"][index] = float(stats[2]) 
                temp["History_" + col + "_Min"][index] = float(stats[3]) 
                temp["History_" + col + "_25%"][index] = float(stats[4]) 
                temp["History_" + col + "_50%"][index] = float(stats[5]) 
                temp["History_" + col + "_75%"][index] = float(stats[6])
                temp["History_" + col + "_Max"][index] = float(stats[7])   
        index += 1
    
    temp["Opp"] = opp
    temp["Team"] = team
    
    
    
    
    #Get the real points they made and bio
    index = 0
    for name in players:
        
        test_game = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] == year) & (data["Week"] == week)]

        # Assuming test_game contains only one row, otherwise this will error out.
        if len(test_game) == 1:
            #print(test_game["Dk_Points"].iloc[0])

            temp.loc[index, "Actaul_Dk_Points"] = test_game["Dk_Points"].iloc[0]
            temp.loc[index, "Month"] = test_game["Month"].iloc[0]
            temp.loc[index, "Year"] = test_game["Year"].iloc[0]
            temp.loc[index, "Week"] = test_game["Week"].iloc[0]
            #temp.loc[index, "Feet"] = test_game["Feet"].iloc[0]
            #temp.loc[index, "Inches"] = test_game["Inches"].iloc[0]
            #temp.loc[index, "Weight(lbs)"] = test_game["Weight(lbs)"].iloc[0]
        else:
            # Handle cases where test_game might have more than one row or no rows.
            print(f"Expected only one row for conditions, but found {len(test_game)} rows.")
        
        index += 1
    
    summary_team_stats["Last_Off_Total_YDS"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Total_YDS/G"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Passing_YDS"] = float(test_game["Last_Off_Passing_YDS"])
    summary_team_stats["Last_Off_Passing_YDS/G"] = float(test_game["Last_Off_Passing_YDS/G"])
    summary_team_stats["Last_Off_Rushing_YDS"] = float(test_game["Last_Off_Rushing_YDS"])
    summary_team_stats["Last_Off_Rushing_YDS/G"] = float(test_game["Last_Off_Rushing_YDS/G"])
    summary_team_stats["Last_Off_Points_PTS"] = float(test_game["Last_Off_Points_PTS"])
    summary_team_stats["Last_Off_PointsPTS/G"] = float(test_game["Last_Off_PointsPTS/G"])
    
    summary_team_stats["Last_Def_Total_YDS"] = float(test_game["Last_Def_Total_YDS"])
    summary_team_stats["Last_Def_Total_YDS/G"] = float(test_game["Last_Def_Total_YDS/G"])
    summary_team_stats["Last_Def_Passing_YDS"] = float(test_game["Last_Def_Passing_YDS"])
    summary_team_stats["Last_Def_Passing_YDS/G"] = float(test_game["Last_Def_Passing_YDS/G"])
    summary_team_stats["Last_Def_Rushing_YDS"] = float(test_game["Last_Def_Rushing_YDS"])
    summary_team_stats["Last_Def_Rushing_YDS/G"] = float(test_game["Last_Def_Rushing_YDS/G"])
    summary_team_stats["Last_Def_Points_PTS"] = float(test_game["Last_Def_Points_PTS"])
    summary_team_stats["Last_Def_PointsPTS/G"] = float(test_game["Last_Def_PointsPTS/G"])
        
        
        
        
        
    
    temp = temp.fillna(0)
    
    
    return temp, summary_team_stats

In [4]:
ready_data, team_data = wr_week_one(clean_WR, "LVR", "LAC", ["", "Adams, Davante"], year = 2022, week = 1)

In [5]:
ready_data

,Name_,Off. Snaps_Num_Average,Off. Snaps_Num_STD,Off. Snaps_Num_Min,Off. Snaps_Num_25%,Off. Snaps_Num_50%,Off. Snaps_Num_75%,Off. Snaps_Num_Max,History_Opp_Off. Snaps_Num_Count,History_Off. Snaps_Num_Average,...,History_Salary_25%,History_Salary_50%,History_Salary_75%,History_Salary_Max,Actaul_Dk_Points,Opp,Team,Month,Year,Week
0,"Adams, Davante",50.24031,21.30536,0.0,46.0,56.0,64.0,87.0,0.0,42.0,...,7100.0,7100.0,7100.0,7100.0,30.1,LAC,LVR,9.0,2022.0,1.0


In [6]:
full_depth = pd.read_csv('full_depth.csv', sep = ";")

In [7]:
full_depth = full_depth[full_depth["Year"] < 2023]
full_depth["Team"] = full_depth["Team"].replace("NO", "NOR").replace("LV", "LVR").replace("KC", "KAN").replace("SF", "SFO").replace("NE", "NWE").replace("GB", "GBN").replace("TB", "TAM").replace("ARZ", "ARI")
full_depth  = full_depth.reset_index(drop = True)


In [31]:
wr_depth = full_depth[(full_depth["Pos"] == "LWR") | (full_depth["Pos"] == "RWR") | (full_depth["Pos"] == "SWR")].reset_index(drop = True)
rb_depth = full_depth[(full_depth["Pos"] == "RB")].reset_index(drop = True)
qb_depth = full_depth[(full_depth["Pos"] == "QB")].reset_index(drop = True)
te_depth = full_depth[(full_depth["Pos"] == "TE")].reset_index(drop = True)
qb_depth


,Pos,1,2,3,4,5,Team,Month,Year
0,QB,"Allen, Josh","Keenum, Case",0,0,0,BUF,12,2022
1,QB,"Tagovailoa, Tua","Bridgewater, Teddy","Thompson, Skylar",0,0,MIA,12,2022
2,QB,"Jones, Mac","Zappe, Bailey",0,0,0,NWE,12,2022
3,QB,"White, Mike","Flacco, Joe","Wilson, Zach",0,0,NYJ,12,2022
4,QB,"Prescott, Dak","Rush, Cooper","Grier, Will",0,0,DAL,12,2022
...,...,...,...,...,...,...,...,...,...
3584,QB,"Rivers, Philip","Whitehurst, Charlie","Sorensen, Brad",0,0,LAC,10,2013
3585,QB,"Palmer, Carson","Stanton, Drew","Lindley, Ryan",0,0,ARI,10,2013
3586,QB,"Kaepernick, Colin","Mccoy, Colt","Daniels, Bj",0,0,SFO,10,2013
3587,QB,"Wilson, Russell","Jackson, Tarvaris",0,0,0,SEA,10,2013


In [27]:
def lwr_data_org(depth, filtered_data):
    #name_options = filtered_data.Name_.unique()
    bad_apple = []
    players = []
    all_dfs = {}
    length = len(depth)
    count = 0
    for index, row in depth.iterrows():
        all_players = [row["1"], row["2"], row["3"], row["4"], row["5"]]
        players = [x for x in all_players if x != "0"]
        #print(players)
        #print(index, "\n")
        try:
            temp_name = players[0]
            team = row["Team"]
            month = row["Month"]
            year = row["Year"]
            #print(team, name, month, year)

            temp = filtered_data[(filtered_data["Name_"] == temp_name) & (filtered_data["Year"] == year) & (filtered_data["Month"] == month)]
            week_list = list(temp.Week)
            oppenant = list(temp.Opp)




            for index_2, week in enumerate(week_list):
                #print(players)
                #print(team, oppenant[index], players, year, week)
                try:
                    ready_data, team_data = wr_week_one(data = filtered_data,
                                                        team =team,
                                                        opp = oppenant[index_2],
                                                        players = players, 
                                                        year = year, 
                                                        week = week)

                    all_dfs[team + "_" + str(week) + "_" + str(year) + "_" + row["Pos"]] = ready_data

                except:
                    #print(team, oppenant[index_2], players, year, week)
                    bad_apple.append(players)
                    #print(players)
                    count += 1
                    #print(count/length)
            if index % 100 == 0:
                print(index/length * 100, "%")
           
        except:
            bad_apple.append(1)
                
    return all_dfs, bad_apple

In [28]:
positions_paths = [clean_QB, clean_WR]#, clean_RB, clean_TE]
depthion_path = [qb_depth, wr_depth]#, rb_depth, te_depth]
saving = ['QB_Master_df.pkl', 'WR_Master_df.pkl', "RB_Master_df.pkl", "TE_Master_df.pkl"]
master_df = {}
count = 0
for data in positions_paths:
    print(count, " Shrimps")
    master_df[count], bad_apple  = lwr_data_org(depthion_path[count], data)
    count += 1

0  Shrimps
0.0 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
2.7862914460852606 %
5.572582892170521 %
8.35887433825578 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for c

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
78.01616049038729 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for co

...

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
61.33600981376157 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for co

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
80.29441284710606 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for co

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

In [ ]:
stop_it_you_popy

In [48]:
QB_Master_df = master_df[0]
WR_Master_df = master_df[1]

In [51]:
import pickle

# Save the dictionary with DataFrames using pickle
with open('QB_Master_df.pkl', 'wb') as pickle_file:
    pickle.dump(QB_Master_df, pickle_file)


In [55]:
import pickle

# Save the dictionary with DataFrames using pickle
with open('WR_Master_df.pkl', 'wb') as pickle_file:
    pickle.dump(WR_Master_df, pickle_file)


In [56]:
positions_paths = [clean_RB, clean_TE]
depthion_path = [rb_depth, te_depth]

saving = ["RB_Master_df.pkl", "TE_Master_df.pkl"]
master_df_2 = {}
count = 0
for data in positions_paths:
    print(count, " Shrimps")
    master_df_2[count], bad_apple  = lwr_data_org(depthion_path[count], data)
    with open(saving[count], 'wb') as pickle_file:
        pickle.dump(master_df_2[count], pickle_file)
    print("Saved to ", saving[count])
    count += 1

0  Shrimps
0.0 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for condi

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
36.36363636363637 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
38.50267379679144 %
40.64171122994652 %
42.780748663101605 %
44.919786096256686 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Ex

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
68.44919786096256 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for co

...

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
58.51493139628733 %
60.53268765133172 %
62.55044390637611 %
64.56820016142049 %
Expected only one row for conditions, but found 0 rows.
Expected only one row fo

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
78.69249394673123 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for co

Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
88.7812752219532 %
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for conditions, but found 0 rows.
Expected only one row for con